# Considerações:

Para realizarmos a conversão das tabelas do Banco de dados, pensamos inicialmente em realizarmos as consultas diretamente através de uma conexão com o banco, porém, achamos mais apropriado extrair as tabelas em CSV e realizarmos a conversão.

Optando por realizar a conversão pelo CSV, encontramos algumas dificuldades em realizar a conversão das consultas como indexação, criar uma relação entre as tabelas. Buscamos solucionar estes problemas extraindo os dados das tabelas primitivas em array e recriando as tabelas em dataframe, alterando algumas colunas das tabelas para facilitar nas consultas

# Trasnformando Tabelas em Dataframes

In [1]:
import pandas as pd
import numpy as np

Tclientes = pd.read_csv("Tclientes.csv")
Tfornecedores = pd.read_csv("Tfornecedores.csv")
Tlogistica = pd.read_csv("Tlogistica.csv ")
Tmateriais = pd.read_csv("Tmateriais.csv")
Tpedidos = pd.read_csv("Tpedidos.csv")
Tproducao = pd.read_csv("Tproducao.csv")
Tprodutos = pd.read_csv("Tprodutos.csv")



In [2]:
# Tabela Clientes
Index = [1,2,3]
Colunas_Clintes = ["ID_Clintes","CPF","Nome", "Endereço","Email"]
Dados_Clintes = Tclientes.values
Clientes = pd.DataFrame(Dados_Clintes, Index ,Colunas_Clintes)

Clientes

,ID_Clintes,CPF,Nome,Endereço,Email
1,1,12345678901,cliente 1,endereço completo,mail@mail.com
2,2,12345678902,cliente 2,endereço completo,mail@mail.com
3,3,12345678903,cliente 3,endereço completo,mail@mail.com


In [3]:
# Tabela Materiais
Index = [1,2,3]
Colunas_materiais =  ["Id_Materiais","Nm_Material", "Qnt_Materiais"]
Dados_Materiais = Tmateriais.values
Materiais = pd.DataFrame(Dados_Materiais,Index,Colunas_materiais)
Materiais

,Id_Materiais,Nm_Material,Qnt_Materiais
1,1,material 1,9
2,2,material 2,10
3,3,material 3,10


In [4]:
# Tabela Fornecedores
Index = [1,2,3]
Colunas_Fornecedores = ["Id_Fornecedores","Id_materiais","Nm_Fornecedor", "Valor_Unitário", "Qnt_Compra"]
Dados_Fornecedores = Tfornecedores.values
Fornecedores = pd.DataFrame (Dados_Fornecedores,Index,Colunas_Fornecedores)
Fornecedores


,Id_Fornecedores,Id_materiais,Nm_Fornecedor,Valor_Unitário,Qnt_Compra
1,1,1,forcenedor material 1,100.0,10
2,2,2,forcenedor material 2,100.0,10
3,3,3,forcenedor material 3,100.0,10


In [5]:
# Tabela Produtos
Index =  [1,2,3]
Colunas_Produtos = ["Id_Produtos","Nm_Produto","Dimensões","Detalhes"]
Dados_Produtos = Tprodutos.values
Produtos = pd.DataFrame(Dados_Produtos,Index,Colunas_Produtos)
Produtos


,Id_Produtos,Nm_Produto,Dimensões,Detalhes
1,1,produto 1,alturaxlarguraxprofundidade/peso,material usado
2,2,produto 2,alturaxlarguraxprofundidade/peso,material usado
3,3,produto 3,alturaxlarguraxprofundidade/peso,material usado


In [6]:
# Tabela Pedidos
Inbdex = [1,2,3]
Colunas =["Id_Pedido","Id_clientes","Id_Produto", "Status"]
Dados = Tpedidos.values
Pedidos = pd.DataFrame(Dados,Index,Colunas)
Pedidos


,Id_Pedido,Id_clientes,Id_Produto,Status
1,1,1,1,Pedido em produção
2,2,2,2,Pedido em rota de entrega
3,3,3,3,Pedido efetuado


In [7]:
# Tabela Logistica
Index = [1]
Colunas_Logistica = ["Id_Logistica","Id_Pedido","PrazoEntrega"]
Dados_Logistica =Tlogistica.values
Logistica = pd.DataFrame(Dados_Logistica,Index,Colunas_Logistica)
Logistica


,Id_Logistica,Id_Pedido,PrazoEntrega
1,1,2,2022-05-30 23:09:00


In [8]:
# Tabela Produção
Index= [1]
Colunas_Produção = ["Id_Produção","Id_Pedido","Id_Materiais","TempoProdução"]
Dados_Produção =Tproducao.values
Produção = pd.DataFrame(Dados_Produção,Index,Colunas_Produção)
Produção

,Id_Produção,Id_Pedido,Id_Materiais,TempoProdução
1,1,1,1,2022-05-30 23:09:00


# Realizando as Consultas

In [9]:
#Visualização dos materiais disponiveis
'''SELECT
nm_material "Material",
qnt_disponivel "Quantidade Disponivel"
FROM mastermobile.materiais;
-- Visualização dos pedidos em produção'''


Materiais.iloc[:,1:]

,Nm_Material,Qnt_Materiais
1,material 1,9
2,material 2,10
3,material 3,10


In [10]:
#Visualização dos pedidos efetuados
'''SELECT
id "Código Pedido",
id_cliente "Código Cliente",
id_produto "Código Produto",
status "Status pedido"
FROM mastermobile.pedidos where status = 'pedido efetuado';'''
Tpedidos[Tpedidos.Status =='Pedido efetuado']

,ID,ID_Cliente,ID_Produto,Status
2,3,3,3,Pedido efetuado


In [11]:
# Visualização dos produtos disponiveis
'''SELECT
nm_produto "Nome do produto",
dimensoes "Dimensões",
detalhes "Detalhes Técnicos"
FROM mastermobile.produtos;'''

Produtos.iloc[:,1:]


,Nm_Produto,Dimensões,Detalhes
1,produto 1,alturaxlarguraxprofundidade/peso,material usado
2,produto 2,alturaxlarguraxprofundidade/peso,material usado
3,produto 3,alturaxlarguraxprofundidade/peso,material usado


In [12]:
#Visualização dos pedidos em produção
'''SELECT
id_pedido "Código Pedido",
id_materiais "Código Material a ser usado",
tempoProducao "Data e hora prevista para conclusão",
pd.status "Status" FROM mastermobile.producao p
INNER JOIN mastermobile.pedidos pd
on pd.id = p.id_pedido
Where pd.status = 'Pedido em produção';'''

Tproducao.join(Tpedidos.Status.loc[Tpedidos.Status == 'Pedido em produção' ])

,ID,ID_Pedido,ID_Materiais,TempoProducao,Status
0,1,1,1,2022-05-30 23:09:00,Pedido em produção


In [13]:
#Visualização dos forncedores cadastrados
"""SELECT
id_materiais "Código Material",
nm_fornecedor "Nome do Fornecedor",
qnt_comprada "Quantidade Total Adquirida",
valor_unitario "Valor unitário pago",
qnt_comprada*valor_unitario "Valor Total da compra"
FROM mastermobile.fornecedores;"""

Fornecedores.iloc[:,1:]

,Id_materiais,Nm_Fornecedor,Valor_Unitário,Qnt_Compra
1,1,forcenedor material 1,100.0,10
2,2,forcenedor material 2,100.0,10
3,3,forcenedor material 3,100.0,10


In [14]:
#Visualização dos clientes cadastrados
'''SELECT
Nome "Nome Cliente",
CPF "CPF",
Endereco "Endereço Completo",
email "Email"
FROM mastermobile.clientes;'''

Clientes.iloc[:,1:]

,CPF,Nome,Endereço,Email
1,12345678901,cliente 1,endereço completo,mail@mail.com
2,12345678902,cliente 2,endereço completo,mail@mail.com
3,12345678903,cliente 3,endereço completo,mail@mail.com


In [15]:
#Quantidade de pedidos em produção
'''SELECT
count(pd.id) "Quantidade de pedidos em Produção"
FROM pedidos pd
where pd.status = "Pedido em Produção";'''
Pedidos.loc[Pedidos.Status == "Pedido em produção"].count()

Id_Pedido      1
Id_clientes    1
Id_Produto     1
Status         1
dtype: int64

In [16]:
#Quantidade de pedidos em rota de entrega
'''SELECT
count(pd.id) "Quantidade de pedidos em Rota de Entrega"
FROM pedidos pd
where pd.status = "Pedido em Rota de Entrega";'''

Pedidos.loc[Pedidos.Status == "Pedido em rota de entrega"].count()

Id_Pedido      1
Id_clientes    1
Id_Produto     1
Status         1
dtype: int64

In [17]:
#Quantidade de pedidos finalizados
'''SELECT
count(pd.id) "Quantidade de pedidos finalizados"
FROM pedidos pd
where pd.status = "Pedido Finalizado";'''

Pedidos.loc[Pedidos.Status == "Pedido finalizado"].count()

Id_Pedido      0
Id_clientes    0
Id_Produto     0
Status         0
dtype: int64

In [18]:
#Quantidade total pedidos
'''SELECT
count(pd.id) "Quantidade de pedidos efetuados"
FROM pedidos pd
where pd.status = "Pedido Efetuado";'''

x= Pedidos.loc[Pedidos.Status == "Pedido efetuado"].size
print('Quantidade de pedidos efetuados:')
x


Quantidade de pedidos efetuados:


4